# Iteration 1

In [1]:
const { Client } = require('@notionhq/client');


async function retrievePage(pageId) {
let notionApiKey = process.env.notion_secret;
let notion = new Client({ auth: notionApiKey });
  try {
    const response = await notion.pages.retrieve({ page_id: pageId });
    return response;
  } catch (error) {
    // Handle errors if needed
    console.error(error);
    throw error;
  }
}

var response = retrievePage('4724abef-8c3e-4736-b2b3-33680bd83929')
console.log(response)

Promise { <pending> }


In [2]:
console.log(response)

Promise {
  {
    object: 'page',
    id: '4724abef-8c3e-4736-b2b3-33680bd83929',
    created_time: '2023-11-22T19:09:00.000Z',
    last_edited_time: '2023-12-08T16:03:00.000Z',
    created_by: { object: 'user', id: 'f4b38e73-d5f6-43c6-99b5-a5cbf1fb60c2' },
    last_edited_by: { object: 'user', id: 'f4b38e73-d5f6-43c6-99b5-a5cbf1fb60c2' },
    cover: null,
    icon: { type: 'external', external: [Object] },
    parent: {
      type: 'database_id',
      database_id: 'b6b0367a-e451-48d3-b985-3464e43c6363'
    },
    archived: false,
    properties: {
      'Project (parent task)': [Object],
      Notes: [Object],
      'total subtask time': [Object],
      'when initiated': [Object],
      'work log': [Object],
      'initiation notes': [Object],
      'Last edited time': [Object],
      'total time': [Object],
      'Roadmap Item': [Object],
      'Achieve by': [Object],
      'Subtasks completion': [Object],
      URL: [Object],
      'Notes from Ginkgo log': [Object],
      'summariz

## 1.1

In [1]:
var {getCurrentTimestamp, loadJsonFile} = require('../src/fileFunctions')
const fs = require('fs');

async function retrievePage(
  pageId, jsonFileName='../data/notion_page', save = true, appendTimestamp = true
  ) {
  const { Client } = require('@notionhq/client');
  const fs = require('fs');
  let notionApiKey = process.env.notion_secret;
  let notion = new Client({ auth: notionApiKey });
    try {
      const response = await notion.pages.retrieve({ page_id: pageId });
      if (save) {
        if (appendTimestamp) {
          const timestamp = getCurrentTimestamp();
          jsonFileName = `${jsonFileName}_${timestamp}`
        }
        await fs.promises.writeFile(`${jsonFileName}.json`, JSON.stringify(response, null, 2));
        console.log(`Saved response to ${jsonFileName}.json`);
      }
      return response;
    } catch (error) {
      // Handle errors if needed
      console.error(error);
      throw error;
    }
}

var response = retrievePage('4724abef-8c3e-4736-b2b3-33680bd83929');


Saved response to ../data/notion_page_2023-12-25_1218.json


# *End of Page*